In [ ]:
!pip install Faker

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import random
from faker import Faker

# Initialize Faker instance for generating fake data
fake = Faker()

# File path for the input Excel file with modified data
file_path = "/content/Synthetic Data.xlsx"

# Load the Excel file with the modified data into a DataFrame
modified_data_df = pd.read_excel(file_path, sheet_name="Modified Data")

# Extract unique business structures and a list of all NAICS codes for random selection
business_structures = modified_data_df["Business Structure"].unique().tolist()
all_naics_codes = list(set(range(111110, 999999)))

# Original columns of the data to ensure the generated synthetic data matches
original_columns = modified_data_df.columns.tolist()

# Define possible ownership percentages and their corresponding weights
ownership_values = [20, 25, 33, 50, 100]
ownership_weights = {20: 5, 25: 5, 33: 5, 50: 35, 100: 50}

# Function to generate revenue data with various trend patterns (growth, decline, etc.)
def generate_revenue():
    """Generates revenue with randomized trend probabilities, allowing for sudden boosts or declines."""

    # Randomly generate the revenue from 2 years ago within a specified range
    rev_2_years_ago = random.randint(50000, 2000000)

    # Randomly select a trend for the business (growth, stable, decline, boost, sharp decline)
    trend = random.choices([
        "growth", "stable", "decline", "boost", "sharp_decline"
    ], weights=[40, 20, 20, 10, 10])[0]

    # Revenue generation logic based on selected trend
    if trend == "growth":
        rev_1_year_ago = random.randint(rev_2_years_ago, rev_2_years_ago + 500000)
        rev_latest = random.randint(rev_1_year_ago, rev_1_year_ago + 500000)
    elif trend == "stable":
        rev_1_year_ago = random.randint(int(0.95 * rev_2_years_ago), int(1.05 * rev_2_years_ago))
        rev_latest = random.randint(int(0.95 * rev_1_year_ago), int(1.05 * rev_1_year_ago))
    elif trend == "decline":
        rev_1_year_ago = random.randint(int(0.85 * rev_2_years_ago), rev_2_years_ago)
        rev_latest = random.randint(int(0.85 * rev_1_year_ago), rev_1_year_ago)
    elif trend == "boost":
        rev_1_year_ago = random.randint(rev_2_years_ago, rev_2_years_ago * 2)
        rev_latest = random.randint(rev_1_year_ago, rev_1_year_ago * 2)
    else:  # Sharp decline
        rev_1_year_ago = random.randint(int(0.5 * rev_2_years_ago), int(0.8 * rev_2_years_ago))
        rev_latest = random.randint(int(0.5 * rev_1_year_ago), int(0.8 * rev_1_year_ago))

    return rev_2_years_ago, rev_1_year_ago, rev_latest, trend

# Function to generate debt data based on the revenue trend
def generate_debt(rev_2_years_ago, rev_1_year_ago, rev_latest, trend):
    """Generates debt based on business revenue trend."""

    # Debt ratio ranges based on the trend (growing, stable, or declining)
    if trend == "growth":
        debt_ratio_range = (0.1, 0.4)
    elif trend == "stable":
        debt_ratio_range = (0.2, 0.5)
    else:  # Declining businesses accumulate more debt
        debt_ratio_range = (0.3, 0.7)

    # Generate debt data for the last 3 years based on the revenue data and trend
    debt_2_years_ago = random.randint(5000, max(5000, int(random.uniform(*debt_ratio_range) * rev_2_years_ago)))
    debt_1_year_ago = random.randint(5000, max(5000, int(random.uniform(*debt_ratio_range) * rev_1_year_ago)))
    debt_latest = random.randint(5000, max(5000, int(random.uniform(*debt_ratio_range) * rev_latest)))

    return debt_2_years_ago, debt_1_year_ago, debt_latest

# Main function to generate synthetic data
def generate_synthetic_data(n):
    synthetic_data = []

    # Loop to generate 'n' synthetic records
    for i in range(n*4):  # Multiply by 4 for diversity in data
        rev_2_years_ago, rev_1_year_ago, rev_latest, trend = generate_revenue()
        debt_2_years_ago, debt_1_year_ago, debt_latest = generate_debt(rev_2_years_ago, rev_1_year_ago, rev_latest, trend)

        # NOI (Net Operating Income) between 10% and 50% of revenue
        noi_2_years_ago = random.randint(int(0.1 * rev_2_years_ago), int(0.5 * rev_2_years_ago))
        noi_1_year_ago = random.randint(int(0.1 * rev_1_year_ago), int(0.5 * rev_1_year_ago))
        noi_latest = random.randint(int(0.1 * rev_latest), int(0.5 * rev_latest))

        # Loan amount is randomly generated within a specified range
        loan_amount = random.randint(5000, 5500000)

        # Net Profit Margin (NPM) calculated as NOI/Revenue * 100
        npm = round((noi_latest / rev_latest) * 100, 2) if rev_latest > 0 else 0

        # Ensure business ownership values sum to 100%
        ownership_value = random.choices(ownership_values, weights=[ownership_weights[v] for v in ownership_values])[0]
        ownership_count = {20: 5, 25: 4, 33: 3, 50: 2, 100: 1}[ownership_value]
        ownership_distribution = [ownership_value] * ownership_count + [0] * (5 - ownership_count)

        # Create a dictionary with all the synthetic data for a single row
        row = {
            "Applicant ID": i + 1,
            "Business Structure": fake.company_suffix(),
            "Country": "US",
            "Location": fake.city() + ", " + fake.state_abbr(),
            "NAICS": random.randint(111110, 999999),
            "Personal Credit Score": random.randint(300, 850),
            "Business Credit Score": random.randint(0, 300),
            "Loan Amount": loan_amount,
            "Annual Revenue (latest year)": rev_latest,
            "Annual Revenue (1 year ago)": rev_1_year_ago,
            "Annual Revenue (2 years ago)": rev_2_years_ago,
            "Business Debt (latest year)": debt_latest,
            "Business Debt (1 year ago)": debt_1_year_ago,
            "Business Debt (2 years ago)": debt_2_years_ago,
            "NOI (latest year)": noi_latest,
            "NOI (1 year ago)": noi_1_year_ago,
            "NOI (2 years ago)": noi_2_years_ago,
            "DSCR (latest year)": round(noi_latest / max(1, debt_latest), 2),
            "DSCR (1 year ago)": round(noi_1_year_ago / max(1, debt_1_year_ago), 2),
            "DSCR (2 years ago)": round(noi_2_years_ago / max(1, debt_2_years_ago), 2),
            "Net Profit Margin": npm,
            "Business Ownership (1)": ownership_distribution[0],
            "Business Ownership (2)": ownership_distribution[1],
            "Business Ownership (3)": ownership_distribution[2],
            "Business Ownership (4)": ownership_distribution[3],
            "Business Ownership (5)": ownership_distribution[4],
            "For Profit": random.choices([True, False], weights=[90, 10])[0],
            "Industry Experience": random.randint(0, 20),
            "Managerial Experience": random.randint(0, 20),
            "Collateral Availability": random.choice([True, False]),
            "Business Expansion": random.choice([True, False]),
            "Acquisition Request": random.choice([True, False]),
            "Working Capital": random.choice([True, False]),
            "Fast Approval": random.choices([True, False], weights=[70, 30])[0],
            "Equipment Purchase or Leasing": random.choice([True, False]),
            "Inventory Purchase": random.choice([True, False]),
            "Real Estate Acquisition or Improvement": random.choice([True, False]),
            "Business Acquisition or Buyout": random.choice([True, False]),
            "Refinancing Existing Debt": random.choice([True, False]),
            "Emergency Funds": random.choice([True, False]),
            "Franchise Financing": random.choice([True, False]),
            "Contract Financing": random.choice([True, False]),
            "Licensing or Permits": random.choice([True, False]),
            "Line of Credit Establishment": random.choice([True, False]),
            "Years in Business": random.randint(0, 20),
        }
        synthetic_data.append(row)

    # Create a DataFrame from the generated data
    synthetic_df = pd.DataFrame(synthetic_data)

    # Filter rows based on valid DSCR values (Debt Service Coverage Ratio)
    synthetic_df = synthetic_df[(synthetic_df["DSCR (latest year)"] >= 0) & (synthetic_df["DSCR (latest year)"] <= 2.5) &
            (synthetic_df["DSCR (1 year ago)"] >= 0) & (synthetic_df["DSCR (1 year ago)"] <= 2.5) &
            (synthetic_df["DSCR (2 years ago)"] >= 0) & (synthetic_df["DSCR (2 years ago)"] <= 2.5)]

    # Ensure all expected columns exist in the synthetic data
    for col in original_columns:
        if col not in synthetic_df.columns:
            synthetic_df[col] = None

    synthetic_df = synthetic_df[original_columns]

    return synthetic_df

# Generate synthetic data for 200,000 rows
synthetic_df = generate_synthetic_data(200000)

# Define the output file path to save the generated synthetic data
new_output_path = "Synthetic_Data_Generated_Colab.xlsx"

# Save the original and generated synthetic data to an Excel file
with pd.ExcelWriter(new_output_path) as writer:
    modified_data_df.to_excel(writer, sheet_name="Original Modified Data", index=False)
    synthetic_df.to_excel(writer, sheet_name="Generated Synthetic Data", index=False)

print("Newly Generated Synthetic Data:")
print(synthetic_df.head())


Newly Generated Synthetic Data:
    Applicant ID Business Structure Country           Location   NAICS  \
2              3                PLC      US  Melendezhaven, MN  291443   
3              4              Group      US      Chrisside, PR  945272   
13            14                Ltd      US  Port Benjamin, CA  500525   
15            16                PLC      US      Lake Mark, NC  266264   
17            18              Group      US    Michaelfurt, OH  919761   

    Business Ownership (1)  Business Ownership (2)  Business Ownership (3)  \
2                      100                       0                       0   
3                       20                      20                      20   
13                     100                       0                       0   
15                      50                      50                       0   
17                     100                       0                       0   

    Business Ownership (4)  Business Ownership (5)  ..

#### Overview

The code provided generates synthetic data for small business loan applicants, simulating key business metrics like revenue, debt, net operating income (NOI), and business ownership structure. This synthetic dataset is primarily created to mimic real-world business conditions in the U.S., and it can be used for financial modeling, machine learning applications, or scenario testing. The synthetic data is generated from an existing Excel file with 50 rows of original data, ensuring that it aligns with the characteristics and trends of real business loan applicants.

---

#### Input Data

The synthetic data is generated based on the data from the provided **Modified Data.xlsx** file. This original file includes 50 rows of business applicant data that serve as the basis for generating the synthetic dataset. Key characteristics of the original data are as follows:

- **Business Structure**: Includes business types such as LLC, Sole Proprietorship, S-Corp, and others. The majority of small businesses seeking SBA loans are LLCs, Sole Proprietorships, or S-Corps.
- **Business Location**: All businesses are located in the U.S., with location data in the format "City, State" (e.g., Denver, CO).
- **NAICS Code**: Each business has a valid NAICS (North American Industry Classification System) code, which is essential for determining loan eligibility.
- **Credit Scores**: Both personal and business credit scores are adjusted within specified ranges, reflecting typical values seen in real business loan applicants.

The synthetic data generated maintains the same structure and follows realistic distributions based on these characteristics.

---

#### Data Considerations

Here are key considerations for the synthetic data based on the provided **Modified Data Considerations**:

1. **Business Structure**:
   - Small businesses typically apply for SBA loans as LLCs, Sole Proprietorships, or S-Corps. Other types such as C-Corps, Partnerships, and Nonprofits are rare.
   
2. **Location**:
   - All businesses are located within the U.S., which aligns with SBA loan eligibility.
   
3. **NAICS Code**:
   - A valid NAICS code is assigned to each business. Some NAICS codes may not be eligible for certain loan products, which is important for real-world loan approval processes.
   
4. **Credit Scores**:
   - Personal credit scores range from 300 to 850, while business credit scores range from 0 to 300. The majority of applicants have business credit scores between 100 and 200.
   
5. **Business Ownership**:
   - Ownership distribution is simulated to reflect the common reality that small businesses often have one or two owners. Ownership values greater than 20% are generated with a higher frequency, but 3+ owners are rare.
   
6. **Revenue and Debt**:
   - Annual revenue values for the last three years are generated to follow realistic patterns, with revenues typically being whole numbers. The Net Profit Margin (NPM) is capped at 15%, and NOI (Net Operating Income) is always less than revenue for each year. Debt values are based on the revenue trends of the business.

7. **Loan Amount**:
   - Loan amounts are whole numbers, typically ranging between $500,000 and $5 million. Small businesses may sometimes request lower amounts, but these values are kept within realistic bounds.

8. **Use of Funds**:
   - Boolean columns are used to indicate specific use cases for the loan, such as working capital, business expansion, equipment leasing, etc. Each column is encoded as `True` or `False`.

9. **Debt Service Coverage Ratio (DSCR)**:
   - DSCR is calculated as the ratio of NOI to business debt, and it falls within a realistic range of 0.4 to 2.0. A higher DSCR (e.g., 1.25 or greater) indicates better business financial health.

10. **Other Features**:
    - Features such as collateral availability, industry experience, managerial experience, and whether the applicant requests fast loan approval are also included, reflecting real-world business loan applications.
    
    - The business ownership columns, ranging from `Business Ownership (1)` to `Business Ownership (5)`, capture how ownership is distributed within the company, with higher weights for 1 or 2 owners.

11. **Challenges for Loan Eligibility**:
    - Factors like the business structure, location, NAICS code, credit scores, DSCR, and loan amount all affect the loan eligibility and loan fit. These considerations are relevant when applying real machine learning models for loan decision-making.

---

#### Functions and Their Descriptions

1. **`generate_revenue()`**
   - **Description**: Generates synthetic revenue data for a business over the past three years (2 years ago, 1 year ago, and the latest year), incorporating different trends such as growth, stability, or decline.
   - **Parameters**: None
   - **Returns**: `rev_2_years_ago`, `rev_1_year_ago`, `rev_latest`, `trend` (revenue for 3 years and the trend type).

2. **`generate_debt(rev_2_years_ago, rev_1_year_ago, rev_latest, trend)`**
   - **Description**: Generates synthetic debt data for the business based on the generated revenue trends. Debt is calculated as a percentage of revenue, with different ranges based on the business's trend.
   - **Parameters**: `rev_2_years_ago`, `rev_1_year_ago`, `rev_latest`, `trend` (revenue data and the trend type).
   - **Returns**: `debt_2_years_ago`, `debt_1_year_ago`, `debt_latest` (debt for 3 years).

3. **`generate_synthetic_data(n)`**
   - **Description**: The main function that generates `n` synthetic business records. For each record, it generates various business attributes such as revenue, debt, NOI, ownership structure, and credit scores. The data is structured to match real-world business conditions.
   - **Parameters**: `n` (number of records to generate).
   - **Returns**: A DataFrame containing `n` synthetic business records.

---

#### Output

- **Synthetic Data File**: The generated synthetic data is saved in an Excel file with two sheets:
  1. **Original Modified Data**: The original 50 rows of data used as the basis for generating synthetic records.
  2. **Generated Synthetic Data**: The newly generated synthetic business data, including all the features described above.

---

#### Dependencies

- **pandas**: For data manipulation and handling the Excel file.
- **random**: To generate random values for various business attributes.
- **Faker**: For generating realistic fake data such as company names and locations.
